In [345]:
import numpy as np
import cv2
import pandas as pd
from matplotlib import pyplot as plt

In [350]:
image = cv2.imread('popcorn_box.jpg')

In [351]:
df = pd.read_csv('coordinates.csv');
coordinates = np.array(df)

P1 = coordinates[0]
P2 = coordinates[1]
P3 = coordinates[2]
P4 = coordinates[3]
P5 = coordinates[4]
P6 = coordinates[5]
P7 = coordinates[6]

In [352]:
x1_p1 = P1
x1_p2 = P2

y1_p1 = P1
y1_p2 = P7

z1_p1 = P1
z1_p2 = P3

In [353]:
cv2.imshow('pop', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [354]:
x2_p1 = P3
x2_p2 = P4

y2_p1 = P3
y2_p2 = P5

z2_p1 = P2
z2_p2 = P4

In [355]:
x3_p1 = P5
x3_p2 = P6

y3_p1 = P4
y3_p2 = P6

z3_p1 = P7
z3_p2 = P5

## Compute Vanishing points

In [356]:
ax1,bx1,cx1 = np.cross(x1_p1,x1_p2) #obtain co-efficients of line equation of line X1
ax2,bx2,cx2 = np.cross(x2_p1,x2_p2)#obtain co-efficients of line equation of line X2
Vx = np.cross([ax1,bx1,cx1],[ax2,bx2,cx2]) #taking cross product of line X1 and X2 gives the point at which the 2 lines meet i.e., the vanishing point

In [357]:
ay1,by1,cy1 = np.cross(y1_p1,y1_p2) #line Y1
ay2,by2,cy2 = np.cross(y2_p1,y2_p2) #line Y2
Vy = np.cross([ay1,by1,cy1],[ay2,by2,cy2]) #intersection of Y1 and Y2 gives vanishing point

In [358]:
az1,bz1,cz1 = np.cross(z1_p1,z1_p2) #line Z1
az2,bz2,cz2 = np.cross(z2_p1,z2_p2) #line Z2
Vz = np.cross([az1,bz1,cz1],[az2,bz2,cz2]) #intersection of Z1 and Z2 gives vanishing point

In [359]:
print("Hence the Vanishing points for this image are :")
print("Along x-axis : ", Vx)
print("Along y-axis : ", Vy)
print("Along z-axis : ", Vz)

Hence the Vanishing points for this image are :
Along x-axis :  [ 1.0340985e+07 -2.6728420e+06  5.2370000e+03]
Along y-axis :  [6928740. 4551624.   -7884.]
Along z-axis :  [2.617140e+06 6.515442e+06 4.704000e+03]


In [360]:
# To obtain VP in homogenous co-ordinate system i.e., of the form (x, y, 1), divide by the third co-efficient
Vx = Vx/Vx[2]
Vy = Vy/Vy[2]
Vz = Vz/Vz[2]

In [361]:
print("Hence the Vanishing points for this image in homogeneous form are :")
print("Along x-axis : ", Vx)
print("Along y-axis : ", Vy)
print("Along z-axis : ", Vz)

Hence the Vanishing points for this image in homogeneous form are :
Along x-axis :  [ 1.97460092e+03 -5.10376551e+02  1.00000000e+00]
Along y-axis :  [-878.83561644 -577.32420091    1.        ]
Along z-axis :  [5.56364796e+02 1.38508546e+03 1.00000000e+00]


## Computing Projection Matrix 

In [362]:
wo = P1 #world origin
ref_x = P2
ref_y = P7
ref_z = P3

In [363]:
# reference axis distance from World Origin
ref_x_dis = np.sqrt(np.sum(np.square(np.array([ref_x]) - wo)))   
ref_y_dis = np.sqrt(np.sum(np.square(np.array([ref_y]) - wo)))   
ref_z_dis = np.sqrt(np.sum(np.square(np.array([ref_z]) - wo)))   

In [364]:
#compute scaling constants evaluated wrt reference points 
ax,_,_,_ = np.linalg.lstsq( (np.array([Vx]) - wo).T , (np.array([ref_x]) - wo).T )
ax = ax[0][0]/ref_x_dis

ay,_,_,_ = np.linalg.lstsq( (np.array([Vy]) - wo).T , (np.array([ref_y]) - wo).T )
ay = ay[0][0]/ref_y_dis

az,_,_,_ = np.linalg.lstsq( (np.array([Vz]) - wo).T , (np.array([ref_z]) - wo).T )
az = az[0][0]/ref_z_dis

C:\Users\ashwi\AppData\Local\Temp\ipykernel_9272\2746440933.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  ax,_,_,_ = np.linalg.lstsq( (np.array([Vx]) - wo).T , (np.array([ref_x]) - wo).T )
C:\Users\ashwi\AppData\Local\Temp\ipykernel_9272\2746440933.py:5: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  ay,_,_,_ = np.linalg.lstsq( (np.array([Vy]) - wo).T , (np.array([ref_y]) - wo).T )
C:\Users\ashwi\AppData\Local\Temp\ipykernel_9272\2746440933.py:8: FutureWarning: `rcond` parameter will change to the default of machi

In [365]:
px = ax*Vx
py = ay*Vy
pz = az*Vz

In [366]:
P = np.zeros([3,4])
P[:,0] = px
P[:,1] = py
P[:,2] = pz
P[:,3] = wo
""" variable description of P : px, py, pz, wo are arranged as columns in P """

' variable description of P : px, py, pz, wo are arranged as columns in P '

In [367]:
P

array([[ 1.06863482e+00, -4.69733874e-01, -7.54500334e-01,
         5.35000000e+02],
       [-2.76210827e-01, -3.08577313e-01, -1.87834933e+00,
         6.48000000e+02],
       [ 5.41190277e-04,  5.34495718e-04, -1.35612523e-03,
         1.00000000e+00]])

## Computing Homograph matrix

In [368]:
Hxy = np.zeros((3,3))
Hyz = np.zeros((3,3))
Hzx = np.zeros((3,3))
Hxy, Hyz, Hzx

(array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]))

In [369]:
Hxy[:,0] = px
Hxy[:,1] = py
Hxy[:,2] = wo

Hyz[:,0] = py
Hyz[:,1] = pz
Hyz[:,2] = wo

Hzx[:,0] = px
Hzx[:,1] = pz
Hzx[:,2] = wo

In [370]:
row,col,_ = image.shape

In [371]:
warp_xy = cv2.warpPerspective(image,Hxy,(row,col),flags=cv2.WARP_INVERSE_MAP)
warp_yz = cv2.warpPerspective(image,Hyz,(row,col),flags=cv2.WARP_INVERSE_MAP)
warp_zx = cv2.warpPerspective(image,Hzx,(row,col),flags=cv2.WARP_INVERSE_MAP)

In [372]:
cv2.imshow("Txy",warp_xy)
cv2.imshow("Tyz",warp_yz)
cv2.imshow("Tzx",warp_zx)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [373]:
cv2.imwrite("Hxy.png", warp_xy)
cv2.imwrite("Hyz.png", warp_yz)
cv2.imwrite("Hzx.png", warp_zx)


True